In [2]:
# import DiCE
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dice_ml
from dice_ml.utils import helpers 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [4]:

data = pd.read_csv('titanic.csv')
data.rename(columns={'Survived': 'class'}, inplace=True)
data['Sex'] = data['Sex'].map({'male':'Male','female':'Female'})
data['Embarked'] = data['Embarked'].map({'S':'Southampton','C':'Cherbourg','Q':'Queenstown'})
data['Pclass'] = data['Pclass'].map({1:'First',2:'Second',3:'Third'})
#data = data.fillna(-999)

data['Relatives'] = data['SibSp'] + data['Parch']

data = data.drop(['PassengerId', 'Name','Ticket','Cabin', 'SibSp', 'Parch'], axis=1)
data = data.dropna()

train_df, test_df = train_test_split(
            data, test_size=0.2, random_state=17)



In [5]:
#data.to_csv("titanic_full.csv")
train_df.to_csv("titanic_train.csv", index=False)
test_df.to_csv("titanic_test.csv", index=False)

In [6]:
print(train_df)

class  Pclass     Sex   Age      Fare     Embarked  Relatives
377      0   First    Male  27.0  211.5000    Cherbourg          2
88       1   First  Female  23.0  263.0000  Southampton          5
296      0   Third    Male  23.5    7.2292    Cherbourg          0
389      1  Second  Female  17.0   12.0000    Cherbourg          0
708      1   First  Female  22.0  151.5500  Southampton          0
..     ...     ...     ...   ...       ...          ...        ...
512      1   First    Male  36.0   26.2875  Southampton          0
491      0   Third    Male  21.0    7.2500  Southampton          0
179      0   Third    Male  36.0    0.0000  Southampton          0
302      0   Third    Male  19.0    0.0000  Southampton          0
784      0   Third    Male  25.0    7.0500  Southampton          0

[569 rows x 7 columns]


NameError: name 'train_pre' is not defined

In [9]:
train_pre = pd.get_dummies(train_df, drop_first=False, columns=["Pclass", "Sex", "Embarked"])

result1 = train_pre.copy()
for feature_name in ['Age', 'Fare', 'Relatives']:
    max_value = data[feature_name].max()
    min_value = data[feature_name].min()
    result1[feature_name] = (
        train_pre[feature_name] - min_value) / (max_value - min_value)
train_pre = result1

test_pre = pd.get_dummies(test_df, drop_first=False, columns=["Pclass", "Sex", "Embarked"])

result2 = test_pre.copy()
for feature_name in ['Age', 'Fare', 'Relatives']:
    max_value = data[feature_name].max()
    min_value = data[feature_name].min()
    result2[feature_name] = (
        test_pre[feature_name] - min_value) / (max_value - min_value)
test_pre = result2
#print(train_pre)

X_train = train_pre.loc[:, train_pre.columns != 'class']
y_train = train_pre.loc[:, train_pre.columns == 'class']
X_test = test_pre.loc[:, test_pre.columns != 'class']
y_test = test_pre.loc[:, train_pre.columns == 'class']


In [50]:
sess = tf.InteractiveSession()

ann_model = keras.Sequential()
ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l1(0.001), activation=tf.nn.relu))
ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
ann_model.fit(X_train, y_train, validation_split=0.20, epochs=100, verbose=0)

#sess = tf.InteractiveSession()

#ann_model = keras.Sequential()
#ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l1(0.001), activation=tf.nn.relu))
#ann_model.add(keras.layers.Dense(2, activation=tf.nn.sigmoid))
#ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
#ann_model.fit(X_train, to_categorical(y_train), validation_split=0.20, epochs=100, verbose=0)

In [51]:
#print(ann_model.evaluate(X_test, to_categorical(y_test), verbose=0)[1])
#print(ann_model.predict(X_test.head(1)))
print(ann_model.evaluate(X_test, y_test, verbose=0)[1])

0.8041958


In [52]:
ann_model.save('titanic_ann.h5')